# Imports

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from yahooquery import Ticker as yq
import requests
from bs4 import BeautifulSoup
import openpyxl
import time

import logging
logger = logging.getLogger('peewee')
logger.setLevel(logging.INFO)  

# ETF Time Series Data

The data for this section was sourced using the Yahoo Finance API.

In [ ]:
# Increase pandas display precision for floats
pd.set_option('display.float_format', '{:.6f}'.format)

def fetch_ohlcv(ticker: str,
                prefix: str,
                start: str,
                end: str,
                auto_adjust: bool = True) -> pd.DataFrame:
    """
    Download daily OHLCV for `ticker` via yfinance,
    then rename columns to '<prefix>_Open', '<prefix>_High', etc.
    """
    df = yf.download(
        tickers=ticker,
        start=start,
        end=end,
        auto_adjust=auto_adjust,
        progress=False,
        group_by='column'
    )[["Open","High","Low","Close","Volume"]]
    # Flatten MultiIndex if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    # Prefix columns
    df = df.rename(columns=lambda c: f"{prefix}_{c}")
    df.index = pd.to_datetime(df.index)
    return df

def build_monthly_te_series(etf: str,
                            benchmark: str,
                            start: str,
                            end: str,
                            decimal_places: int = 6) -> pd.DataFrame:
    """
    1) Fetch daily OHLCV for ETF and benchmark.
    2) Resample ETF daily OHLCV to month-end ('ME'):
         - ETF_Open   = first open of the month
         - ETF_High   = max high in the month
         - ETF_Low    = min low in the month
         - ETF_Close  = last close of the month
         - ETF_Volume = total monthly volume
    3) Resample benchmark daily Close to month-end:
         - Benchmark_Close = last close of the month
    4) Compute monthly returns:
         - ETF_Return       = (ETF_Close_t / ETF_Close_{t-1}) - 1
         - Benchmark_Return = (Benchmark_Close_t / Benchmark_Close_{t-1}) - 1
    5) Compute benchmark volatility within each month:
         - daily_ret_t = (Benchmark_Close_t / Benchmark_Close_{t-1}) - 1
         - Benchmark_Volatility = std(daily_ret) over each calendar month
    6) Compute Tracking Error:
         - Tracking_Error  = ETF_Return − Benchmark_Return
    7) Merge all into one DataFrame indexed by month (YYYY-MM), round to desired precision, and drop NaNs.
    """
    # 1) Daily data
    etf_daily   = fetch_ohlcv(etf,       "ETF",       start, end)
    bench_daily = fetch_ohlcv(benchmark, "Benchmark", start, end)

    # 2) Monthly ETF OHLCV
    monthly_etf = etf_daily.resample('ME').agg({
        'ETF_Open':   'first',
        'ETF_High':   'max',
        'ETF_Low':    'min',
        'ETF_Close':  'last',
        'ETF_Volume': 'sum',
    })

    # 3) Monthly benchmark Close
    monthly_bench = bench_daily[['Benchmark_Close']].resample('ME').last()

    # 4) Monthly returns
    monthly_etf['ETF_Return']         = monthly_etf['ETF_Close'].pct_change()
    monthly_bench['Benchmark_Return'] = monthly_bench['Benchmark_Close'].pct_change()

    # 5) Benchmark volatility: std of daily returns per month
    daily_bench_ret = bench_daily['Benchmark_Close'].pct_change()
    bench_vol = daily_bench_ret.resample('ME').std().rename('Benchmark_Volatility')
    monthly_bench = monthly_bench.join(bench_vol)

    # 6) Tracking error
    monthly_bench['Tracking_Error']          = (monthly_etf['ETF_Return'] - monthly_bench['Benchmark_Return'])

    # 7) Combine
    df_monthly = monthly_etf.join(
        monthly_bench[[
            'Benchmark_Return',
            'Benchmark_Volatility',
            'Tracking_Error'
        ]]
    ).dropna()

    # Format index to year-month period
    df_monthly.index = df_monthly.index.to_period('M')

    # Round numeric columns to specified decimal places
    return df_monthly.round(decimal_places)

In [3]:
# Function validation with QQQ ETF and NDX benchmark
df_monthly_QQQ = build_monthly_te_series(
    etf="QQQ",
    benchmark="^NDX",
    start="2004-12-31",
    end="2024-12-31"
)
display(df_monthly_QQQ.head())
display(df_monthly_QQQ.tail())

,ETF_Open,ETF_High,ETF_Low,ETF_Close,ETF_Volume,ETF_Return,Benchmark_Return,Benchmark_Volatility,Tracking_Error
Date,,,,,,,,,
2005-01,34.282428,34.453456,31.178290,31.982122,2234773200,-0.063126,-0.062605,0.010865,-0.000522
2005-02,32.059075,32.905666,31.383544,31.828205,1765610600,-0.004813,-0.005666,0.010056,0.000853
2005-03,31.956476,32.743178,30.733606,31.272375,2133973700,-0.017463,-0.018855,0.008751,0.001391
2005-04,31.486132,31.699925,29.373952,29.912689,2345433600,-0.043479,-0.041645,0.012613,-0.001834
2005-05,30.015308,32.726082,29.835730,32.563622,1881340000,0.088622,0.085755,0.006330,0.002867


,ETF_Open,ETF_High,ETF_Low,ETF_Close,ETF_Volume,ETF_Return,Benchmark_Return,Benchmark_Volatility,Tracking_Error
Date,,,,,,,,,
2024-08,469.112327,482.815036,421.073490,473.597015,908585600,0.011039,0.010960,0.016216,0.000079
2024-09,470.544292,491.619134,445.674590,486.012848,694796200,0.026216,0.024831,0.013263,0.001385
2024-10,485.644438,499.236836,475.387832,481.810638,660176300,-0.008646,-0.008488,0.010030,-0.000159
2024-11,483.453661,513.406942,482.208926,507.591461,567729800,0.053508,0.052284,0.010318,0.001224
2024-12,508.856114,536.877522,507.143406,514.264648,598293500,0.013147,0.012743,0.012896,0.000404


# ETF Static Data

The data for this section was sourced using both Yahoo Finance and FinViz.

In [4]:
# Use a real browser user‐agent to avoid being blocked
_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

def extract_etf_features(tickers: list[str], benchmarks: list[str]) -> pd.DataFrame:
    """
    Extract core static ETF features for each ticker, and attach its benchmark index:
      1) ETF Ticker
      2) Benchmark Index (user-supplied)
      3) Net Assets (USD)
      4) NAV (USD)
      5) P/E Ratio (trailing)
      6) Dividend Yield (%)
      7) Beta (5Y Monthly)           – from Finviz snapshot
      8) Expense Ratio (%)           – yfinance
      9) Fund Family                 – yfinance
     10) Fund Category               – yfinance
     11) Inception Date             – yfinance
     12) % in Top 10 Holdings       – yahooquery
     13) Total Holdings             – from Finviz snapshot
     14) Optionable (True/False)    – from Finviz snapshot
     15) Shortable (True/False)     – from Finviz snapshot

    Parameters
    ----------
    tickers : list of str
        ETF tickers to fetch features for.
    benchmarks : list of str
        Corresponding benchmark index name for each ticker.

    Returns
    -------
    pd.DataFrame
        DataFrame with one row per ETF and columns for each feature.
    """
    if len(tickers) != len(benchmarks):
        raise ValueError("tickers and benchmarks must be the same length")

    rows = []
    for t, idx in zip(tickers, benchmarks):
        # 1–4 & 6–9 via yfinance
        info = yf.Ticker(t).info
        
        net_assets     = info.get("netAssets")
        nav            = info.get("navPrice")
        pe_ratio       = info.get("trailingPE")
        
        dy = info.get("dividendYield")
        dividend_yield = dy * 100 if dy is not None else None
        
        er = info.get("expenseRatio") or info.get("netExpenseRatio")
        expense_ratio = er * 100 if er is not None else None
        
        fund_family   = info.get("fundFamily")
        fund_category = info.get("category")
        
        inc_ts = info.get("fundInceptionDate")
        inception_date = pd.to_datetime(inc_ts, unit="s") if inc_ts else None

        # 10 via yahooquery
        yq_t  = yq(t)
        hi    = yq_t.fund_holding_info.get(t, {}) or {}
        holds = hi.get("holdings", [])
        df_h  = pd.DataFrame(holds)
        if not df_h.empty and "holdingPercent" in df_h.columns:
            df_h["holdingPercent"] = pd.to_numeric(df_h["holdingPercent"], errors="coerce")
            pct_top10 = float(df_h.nlargest(10, "holdingPercent")["holdingPercent"].sum()) / 100.0
        else:
            pct_top10 = None

        # 5, 11–13 via Finviz
        finviz_url = f"https://finviz.com/quote.ashx?t={t}"
        html = requests.get(finviz_url, headers=_HEADERS, timeout=10).text
        snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]
        flat = snap.values.flatten()
        snap_map = {flat[i]: flat[i + 1] for i in range(0, len(flat), 2)}

        beta = None
        if "Beta (5Y Monthly)" in snap_map:
            beta = float(snap_map["Beta (5Y Monthly)"])
        elif "Beta" in snap_map:
            beta = float(snap_map["Beta"])

        total_holdings = None
        if "Total Holdings" in snap_map:
            th = snap_map["Total Holdings"].replace(",", "")
            total_holdings = int(th)

        optionable = shortable = None
        optshort = snap_map.get("Option/Short", "")
        if " / " in optshort:
            op, sh = [x.strip() for x in optshort.split(" / ", 1)]
            optionable = (op == "Yes")
            shortable  = (sh == "Yes")

        rows.append({
            "ETF_Ticker":            t,
            "Benchmark_Index":       idx,
            "Net_Assets_USD":        net_assets,
            "NAV_USD":               nav,
            "PE_Ratio":              pe_ratio,
            "Dividend_Yield_pct":    dividend_yield,
            "Beta":                  beta,
            "Expense_Ratio_pct":     expense_ratio,
            "Fund_Family":           fund_family,
            "Fund_Category":         fund_category,
            "Inception_Date":        inception_date,
            "Pct_in_Top10_Holdings": pct_top10,
            "Total_Holdings":        total_holdings,
            "Optionable":            optionable,
            "Shortable":             shortable
        })

    return pd.DataFrame(rows)


In [5]:
# Example test for QQQ
df_static_qqq = extract_etf_features(['QQQ'], ['^NDX'])
display(df_static_qqq.T) # Transposed for better readability

C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


,0
ETF_Ticker,QQQ
Benchmark_Index,^NDX
Net_Assets_USD,333553435000.000000
NAV_USD,530.820000
PE_Ratio,32.577915
Dividend_Yield_pct,58.000000
Beta,1.180000
Expense_Ratio_pct,20.000000
Fund_Family,Invesco
Fund_Category,Large Growth


# Macro Data

VIX Data was sourced from Yahoo Finance and Michigan Consumer Sentiment Index data was sourced from the FRED website.

In [6]:
def macro_data(csv_path: str,
                                start: str = None,
                                end: str = None) -> pd.DataFrame:
    """
    Loads University of Michigan Consumer Sentiment from a CSV and
    VIX daily closes from yfinance, then returns a DataFrame
    indexed by Year-Month (PeriodIndex) with columns:
      - vix
      - michigan_consumer_sentiment_index

    Parameters
    ----------
    csv_path : str
        Path to the UMCSENT.csv file (must have columns
        'observation_date' & 'UMCSENT').
    start : str, optional
        yfinance start date ('YYYY-MM-DD'). Defaults to first sentiment month.
    end : str, optional
        yfinance end date. Defaults to today.
    """
    # 1) Load & prepare monthly Michigan sentiment
    sent = (
        pd.read_csv(csv_path, parse_dates=['observation_date'])
          .set_index('observation_date')
          .sort_index()[['UMCSENT']]
          .rename(columns={'UMCSENT': 'michigan_consumer_sentiment_index'})
          .to_period('M')
    )

    # 2) Determine VIX download range
    if start is None:
        start = sent.index.min().to_timestamp().strftime('%Y-%m-%d')
    if end is None:
        end = pd.Timestamp.today().strftime('%Y-%m-%d')

    # 3) Download VIX daily closes and ensure single-level column 'vix'
    vix = (
        yf.download('^VIX', start=start, end=end, progress=False)[['Close']]
    )
    vix.columns = ['vix']  # rename and flatten
    vix.index = pd.to_datetime(vix.index)

    # 4) Resample to month-end and convert to PeriodIndex
    vix_m = vix.resample('M').last().to_period('M')

    # 5) Merge into one monthly table
    df = pd.concat([vix_m, sent], axis=1).dropna()

    return df

In [7]:
# Output for Michigan Consumer Sentiment and VIX
macro_df = macro_data(
    csv_path='UMCSENT.csv',
    start='2005-01-01',
    end='2024-12-31'
)
display(macro_df.head())

# Save the macro data to an Excel file
macro_df.to_excel('macro_data.xlsx')

C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\1807163659.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download('^VIX', start=start, end=end, progress=False)[['Close']]
C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\1807163659.py:44: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vix_m = vix.resample('M').last().to_period('M')


,vix,michigan_consumer_sentiment_index
2005-01,12.820000,95.500000
2005-02,12.080000,94.100000
2005-03,14.020000,92.600000
2005-04,15.310000,87.700000
2005-05,13.290000,86.900000


# ETF Data Output

### Single ETF Test

In [8]:
# Use one month earlier to include Jan 2005
qqq_ts = build_monthly_te_series("QQQ", "^NDX", "2004-12-01", "2024-12-31")
qqq_static = extract_etf_features(["QQQ"], ["^NDX"]).iloc[0]

# Repeat static row across all dates
qqq_static_df = pd.DataFrame([qqq_static.to_dict()]*len(qqq_ts), index=qqq_ts.index)

# Combine
qqq_df_combined = pd.concat([qqq_ts, qqq_static_df], axis=1)

# Reorder so 'ETF_Ticker' is first
cols = ['ETF_Ticker'] + [c for c in qqq_df_combined.columns if c != 'ETF_Ticker']
qqq_df_combined = qqq_df_combined[cols]

# Display combined DataFrame
display(qqq_df_combined.head())

# Save to Excel
output_file = "qqq_data.xlsx"
qqq_df_combined.to_excel(output_file)
print(f"Saved combined dataset to {output_file}")

C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


,ETF_Ticker,ETF_Open,ETF_High,ETF_Low,ETF_Close,ETF_Volume,ETF_Return,Benchmark_Return,Benchmark_Volatility,Tracking_Error,...,Dividend_Yield_pct,Beta,Expense_Ratio_pct,Fund_Family,Fund_Category,Inception_Date,Pct_in_Top10_Holdings,Total_Holdings,Optionable,Shortable
Date,,,,,,,,,,,,,,,,,,,,,
2005-01,QQQ,34.282459,34.453487,31.178304,31.982122,2234773200,-0.063126,-0.062605,0.010865,-0.000521,...,58.000000,1.180000,20.000000,Invesco,Large Growth,1999-03-10,0.005014,101,True,True
2005-02,QQQ,32.059087,32.905655,31.383518,31.828196,1765610600,-0.004813,-0.005666,0.010056,0.000853,...,58.000000,1.180000,20.000000,Invesco,Large Growth,1999-03-10,0.005014,101,True,True
2005-03,QQQ,31.956457,32.743198,30.733607,31.272354,2133973700,-0.017464,-0.018855,0.008751,0.001391,...,58.000000,1.180000,20.000000,Invesco,Large Growth,1999-03-10,0.005014,101,True,True
2005-04,QQQ,31.486138,31.699916,29.373945,29.912682,2345433600,-0.043478,-0.041645,0.012613,-0.001833,...,58.000000,1.180000,20.000000,Invesco,Large Growth,1999-03-10,0.005014,101,True,True
2005-05,QQQ,30.015303,32.726089,29.835725,32.563595,1881340000,0.088622,0.085755,0.006330,0.002867,...,58.000000,1.180000,20.000000,Invesco,Large Growth,1999-03-10,0.005014,101,True,True


Saved combined dataset to qqq_data.xlsx


### All ETFs Data

In [9]:
# Define ETF–Benchmark pairs 
etf_benchmark_pairs = [
    ("SPY", "^GSPC"), ("DIA", "^DJI"), ("QQQ", "^NDX"), ("MDY", "^SP400"), ("RSP", "^SPXEW"),
    ("IVV", "^GSPC"), ("IJR", "^SP600"), ("IJH", "^SP400"), ("IWM", "^RUT"), ("IWB", "^RUI"),
    ("IWV", "^RUA"), ("IWF", "^RLG"), ("IWD", "^RLV"), ("IWO", "^RUO"), ("IWN", "^RUJ"),
    ("VTI", "^W5000"), ("ONEQ", "^IXIC"), ("IBB", "^NBI"), ("IDU", "^DJU"),
    ("IYT", "^DJT"), ("XLB", "^SP500-15"), ("XLI", "^SP500-20"), ("XLY", "^SP500-25"),
    ("XLP", "^SP500-30"), ("XLV", "^SP500-35"), ("XLF", "^SP500-40"), ("XLK", "^SP500-45"),
    ("XLU", "^SP500-55"), ("XLE", "^GSPE"), ("SMH", "^SOX"), ("SOXX", "^SOX"), ("OIH", "^OSX"),
    ("OEF", "^OEX"), ("IYY", "^DJUS"), ("IYG", "^BKX")
]

# Loop over each pair 
all_dfs = []

for i, (etf, bench) in enumerate(etf_benchmark_pairs, 1):
    print(f"Processing {i}/{len(etf_benchmark_pairs)}: {etf} ~ {bench}")

    try:
        # 1-month buffer at start
        ts_df = build_monthly_te_series(etf, bench, start="2004-12-01", end="2024-12-31")
        static_row = extract_etf_features([etf], [bench]).iloc[0]
        static_df = pd.DataFrame([static_row.to_dict()] * len(ts_df), index=ts_df.index)

        df_combined = pd.concat([ts_df, static_df], axis=1)
        df_combined = df_combined[["ETF_Ticker"] + [c for c in df_combined.columns if c != "ETF_Ticker"]]
        all_dfs.append(df_combined)

        # Delay between requests
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ Failed to process {etf}: {e}")
        continue

# Concatenate & Export 
etf_data = pd.concat(all_dfs)
etf_data.index.name = "Date"
display(etf_data.head())
etf_data.to_excel("etf_data.xlsx")

Processing 1/35: SPY ~ ^GSPC


C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


Processing 2/35: DIA ~ ^DJI


C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


Processing 3/35: QQQ ~ ^NDX


...

Processing 34/35: IYY ~ ^DJUS


C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


Processing 35/35: IYG ~ ^BKX


C:\Users\aatis\AppData\Local\Temp\ipykernel_26232\200606412.py:79: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  snap = pd.read_html(html, attrs={"class": "snapshot-table2"})[0]


,ETF_Ticker,ETF_Open,ETF_High,ETF_Low,ETF_Close,ETF_Volume,ETF_Return,Benchmark_Return,Benchmark_Volatility,Tracking_Error,...,Dividend_Yield_pct,Beta,Expense_Ratio_pct,Fund_Family,Fund_Category,Inception_Date,Pct_in_Top10_Holdings,Total_Holdings,Optionable,Shortable
Date,,,,,,,,,,,,,,,,,,,,,
2005-01,SPY,82.933676,83.070128,79.392844,80.614067,1184211500,-0.022420,-0.025290,0.006447,0.002870,...,121.000000,1.010000,9.450000,SPDR State Street Global Advisors,Large Blend,1993-01-22,0.003573,504,True,True
2005-02,SPY,80.675479,83.008700,80.573141,82.299217,1025608400,0.020904,0.018903,0.006730,0.002001,...,121.000000,1.010000,9.450000,SPDR State Street Global Advisors,Large Blend,1993-01-22,0.003573,504,True,True
2005-03,SPY,82.428789,84.086674,79.622477,80.793709,1330548800,-0.018293,-0.019118,0.006339,0.000825,...,121.000000,1.010000,9.450000,SPDR State Street Global Advisors,Large Blend,1993-01-22,0.003573,504,True,True
2005-04,SPY,81.252579,81.684066,77.773181,79.280006,1633318500,-0.018735,-0.020109,0.009503,0.001373,...,121.000000,1.010000,9.450000,SPDR State Street Global Advisors,Large Blend,1993-01-22,0.003573,504,True,True
2005-05,SPY,79.499193,82.362160,78.629339,81.834778,1334647300,0.032225,0.029952,0.006390,0.002273,...,121.000000,1.010000,9.450000,SPDR State Street Global Advisors,Large Blend,1993-01-22,0.003573,504,True,True
